To complete this assignment, fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`, as well as your name below.

To make sure everything runs as expected, do the following
- **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart)
- **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

A good introduction to Jupyter notebooks is [here](https://realpython.com/jupyter-notebook-introduction/).


In [1]:
NAME = "Joy Zhuge"

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Data-structures" data-toc-modified-id="Data-structures-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data structures</a></span></li><li><span><a href="#Parser-states-and-actions" data-toc-modified-id="Parser-states-and-actions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parser states and actions</a></span></li><li><span><a href="#Training-an-action-classifier" data-toc-modified-id="Training-an-action-classifier-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Training an action classifier</a></span><ul class="toc-item"><li><span><a href="#Generate-training-data" data-toc-modified-id="Generate-training-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Generate training data</a></span></li><li><span><a href="#Compute-features" data-toc-modified-id="Compute-features-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Compute features</a></span></li><li><span><a href="#Fit-classifier" data-toc-modified-id="Fit-classifier-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Fit classifier</a></span></li></ul></li><li><span><a href="#Using-the-classifier-to-parse" data-toc-modified-id="Using-the-classifier-to-parse-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Using the classifier to parse</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Evaluation</a></span></li><li><span><a href="#Bonus:-Feature-engineering" data-toc-modified-id="Bonus:-Feature-engineering-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Bonus: Feature engineering</a></span></li></ul></div>

## Introduction


In this assignment we will implement a transition-based dependency parser, using machine learning to choose the best transition at each step.

As in the previous assignment, there are spaces below for you to both write code and short answers. In some places, there are tests to check your work, though passing tests does not guarantee full credit. I recommend moving sequentially from top to bottom, getting each step working before moving on to the next.

This assignment will use a number of Python libraries, listed in the next cell. If you haven't already installed these, you can do so by running this command in this directory: `pip install -r requirements.txt`. Minor variants in the version numbers shouldn't affect things much.



In [2]:
# imports
from collections import Counter, defaultdict, deque
import numpy as np
from numpy import array as npa
import os
import math
import re
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from tqdm import tqdm

## Data structures

First, we need a data structure to store dependency trees. We'll use the one below. For simplicity, we'll assume unlabeled dependency trees.

Note that we need the word position in the sentence to deal with the case where a word appears more than once.

The children are stored as a list of other DTree objects.

In [3]:
class DTree:
    def __init__(self, word, position):
        self.word = word
        self.position = position
        self.children = []
        
    def add_child(self, child):
        self.children.append(child)
        
    def __repr__(self):
        # for pretty printing a single node
        return '%s-%d' % (self.word, self.position)
    
    def print(self, indent=0):
        # for printing a node and all its descendents
        print(' '*indent + '-'*min(2, indent) + str(self))
        for c in self.children:
            c.print(indent + 2)

Here's an example tree for our example "Book me the morning flight"

In [4]:
def example_tree():
    root = DTree('ROOT', 0)
    book = DTree('Book', 1)
    me = DTree('me', 2)
    the = DTree('the', 3)
    morning = DTree('morning', 4)
    flight = DTree('flight', 5)

    root.add_child(book)
    book.add_child(me)
    book.add_child(flight)
    flight.add_child(the)
    flight.add_child(morning)
    return root

root = example_tree()
root.print()


ROOT-0
  --Book-1
    --me-2
    --flight-5
      --the-3
      --morning-4


Output should be:
    
```
ROOT-0
  --Book-1
    --me-2
    --flight-5
      --the-3
      --morning-4
```

Given a `DTree` defined above, we will often want to extract the parents of each word. Complete the method below, which takes in a DTree and returns a `dict` that maps each `DTree` node to its parent node.

In [5]:
def get_parents(node, node2parent):
    """
    Create a dict of string->string indicating who the parent of each node is in 
    a dependency tree.
    
    Args:
      node..........a DTree node
      node2parent...a dict from str to str, where the value is the parent of the key.
      You can use str(node) to get the string representation of each node (word-position),
      which should be used for both keys and values
      
    Returns:
      nothing. node2parent is passed by reference and is used as the result, to make it 
      easier to implement this recursively.
    """
    
    # YOUR CODE HERE
    
    len_c=len(node.children)
    for i in range(len_c):
        chil=node.children[i]
        node2parent[str(chil)]=str(node)
        get_parents(chil,node2parent)
#     raise NotImplementedError()
    return node2parent

node2parent = {}
get_parents(example_tree(), node2parent)

{'Book-1': 'ROOT-0',
 'me-2': 'Book-1',
 'flight-5': 'Book-1',
 'the-3': 'flight-5',
 'morning-4': 'flight-5'}

Expected output:

```
{'Book-1': 'ROOT-0',
 'me-2': 'Book-1',
 'flight-5': 'Book-1',
 'the-3': 'flight-5',
 'morning-4': 'flight-5'}
```

In [6]:
# test tree: ROOT -> c1 -> c2
root = DTree('ROOT', 0)
c1 = DTree('C1', 1)
c2 = DTree('C2', 2)
root.add_child(c1)
c1.add_child(c2)

node2parent_test = {}
parents_test = get_parents(root, node2parent)

assert parents_test[str(c1)] == str(root)
assert parents_test[str(c2)] == str(c1)

In [7]:
# new version
# test tree: ROOT -> c1 -> c2
root = DTree('ROOT', 0)
c1 = DTree('C1', 1)
c2 = DTree('C2', 2)
root.add_child(c1)
c1.add_child(c2)

node2parent_test = {}
get_parents(root, node2parent_test)

assert node2parent_test[str(c1)] == str(root)
assert node2parent_test[str(c2)] == str(c1)



## Parser states and actions

Next, we need a method to construct a dependency tree given a sentence and a list of dependency parse operators. Recall that the operators are:

- **SHIFT**: move a word from the input buffer to the stack
- **ARC_L**: left arc; add a dependency link from the last element of the stack to the second-to-last element of the stack. remove the second-to-last element of the stack.
- **ARC_R**: right arc; add a dependency link from the second-to-last element of the stack to the last element of the stack. remove the last element of the stack.

e.g., 

**Input:**
- ['book', 'me', 'the', 'morning', 'flight']
- ['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_R', 'ARC_R']

**Output:**

The tree

```
ROOT
  --Book
    --me
    --flight
      --the
      --morning
```

To handle this, we'll use an additional data structure that keeps track of the parser state.

We'll rely on the [deque](https://docs.python.org/3/library/collections.html#collections.deque) class, which is a double-ended queue for efficient insertion and removal from the head/tail of a list.


In [8]:
# example of deque usage.

stack = deque() 
# push elements onto stack
stack.append('a')
stack.append('b')
stack.append('c')
 
print('Initial stack:')
print(stack)

print('\nWe can pop the last element with .pop()')
print(stack.pop())
print('leaving')
print(stack)

print('\nLike a normal list, we can "peek" at any element we like without removing it')
print('peek', stack[-1])
print('stack is still')
print(stack)
# print(stack.pop())
# print(stack.pop())

print('\nsince it is double-ended, we can also use it like a queue and pop from the left:')
print(stack.popleft())
print('this will be useful for storing words in the sentence')
print(stack)

Initial stack:
deque(['a', 'b', 'c'])

We can pop the last element with .pop()
c
leaving
deque(['a', 'b'])

Like a normal list, we can "peek" at any element we like without removing it
peek b
stack is still
deque(['a', 'b'])

since it is double-ended, we can also use it like a queue and pop from the left:
a
this will be useful for storing words in the sentence
deque(['b'])


In [9]:
class State:
    """
    stack....stack of DTree nodes (dequeue)
    input....input DTree nodes (dequeue)
    root.....reference to the ROOT DTree node
    
    Note that we initialize the State with a ROOT node, then 
    push the words in the sentence onto the input deque as DTree objects.
    """
    def __init__(self, sentence):
        self.stack = deque()
        self.stack.append(DTree('ROOT', 0))
        self.input = deque([DTree(w, i) for i, w in enumerate(sentence)])
        self.root = self.stack[0]        

    def __repr__(self):
        return 'input: %s' % str([str(i) for i in self.input]) + \
               '\nstack: %s' % str([str(i) for i in self.stack])

# we'll use this example sentence a lot.
EXAMPLE_SENTENCE = ['Book', 'me', 'the', 'morning', 'flight']
state = State(EXAMPLE_SENTENCE)
state

input: ['Book-0', 'me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0']

With the `State` class, we can now implement `apply_action`. This function will modify the state based on the provided action. We'll assume the action is a string in `['SHIFT', 'ARC_L', 'ARC_R']`.

In [10]:
def apply_action(state, action):
    """
    Modify the state based on the provided action.
    Args:
      state....a State object
      action...a string in ['SHIFT', 'ARC_L', 'ARC_R']
    Returns:
      nothing. state is modified.
    """
    if action == 'SHIFT':
        # YOUR CODE HERE
        state.stack.append(state.input[0])
        state.input.popleft()
        #raise NotImplementedError()
    elif action == 'ARC_L':
        # YOUR CODE HERE
        last=state.stack.pop()
        second_last=state.stack.pop()
        last.add_child(second_last)
        state.stack.append(last)
        #raise NotImplementedError()
    elif action == 'ARC_R':
        # YOUR CODE HERE
        last=state.stack.pop()
        state.stack[-1].add_child(last)
        #raise NotImplementedError()
    else:
        raise Exception('Unknown action: %s' % action)
    

state = State(EXAMPLE_SENTENCE)
print('Initial state')
print(state)

print('\nDo SHIFT')
apply_action(state, 'SHIFT')
print(state)

print('\nDo SHIFT')
apply_action(state, 'SHIFT')
print(state)

print('\nDo ARC_L')
apply_action(state, 'ARC_L')
print(state)

print('\nDo ARC_R')
apply_action(state, 'ARC_R')
print(state)

print('\ncurrent tree')
state.root.print()

Initial state
input: ['Book-0', 'me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0']

Do SHIFT
input: ['me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0']

Do SHIFT
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0', 'me-1']

Do ARC_L
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'me-1']

Do ARC_R
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0']

current tree
ROOT-0
  --me-1
    --Book-0


Expected output:
    
```
Initial state
input: ['Book-0', 'me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0']

Do SHIFT
input: ['me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0']

Do SHIFT
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0', 'me-1']

Do ARC_L
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'me-1']

Do ARC_R
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0']

current tree
ROOT-0
  --me-1
    --Book-0
```

In [11]:
# Test on our running example.
EXAMPLE_ACTIONS = ['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_R', 'ARC_R']
state_test = State(EXAMPLE_SENTENCE)
nodes_test = state_test.input.copy()

for action in EXAMPLE_ACTIONS:
    apply_action(state_test, action)  

# test that the final tree looks like this:
# ROOT-0
#   --Book-1
#     --me-2
#     --flight-5
#       --the-3
#       --morning-4

assert state_test.root.word == 'ROOT'
book = state_test.root.children[0]
assert book  == nodes_test[0]

me = book.children[0]
assert me == nodes_test[1]

flight = book.children[1]
assert flight == nodes_test[4]

# note that "the" is the first child of flight and morning is the second, due to the order in which they
# are added.
the = flight.children[1]
assert the == nodes_test[2]

morning = flight.children[0]
assert morning == nodes_test[3]

We will need a way to determine when there are no more actions left to perform. I.e., when should we stop calling apply_action? What properties should the State have when we should stop? Implement the method below to do so.

In [12]:
# There's no more action left when stack only has ROOT-0 left, and input is None
def is_finished(state):
    # YOUR CODE HERE
    if len(state.stack)==1 and len(state.input)==0:
       return True
    else:
        return False
    #raise NotImplementedError()

In [13]:
state_test = State(EXAMPLE_SENTENCE)
nodes_test = state_test.input.copy()

# apply all but the last action and confirm is_finished is False
for action in EXAMPLE_ACTIONS[:-1]:
    apply_action(state_test, action)  
    assert is_finished(state_test) == False

# apply final action
apply_action(state_test, EXAMPLE_ACTIONS[-1])
assert is_finished(state_test) == True

Below is a helper function to apply all the actions to a given sentence.

In [14]:
def apply_actions(sentence, actions, verbose=False):
    state = State(sentence)
    for action in actions:

        if is_finished(state): # stop if needed
            break
            
        apply_action(state, action)
        if verbose:
            print('\nstate after action %s' % action)
            print(state)
    return state

# note that in the final answer, "morning" and "the" are swapped due to the order the dependencies are added.
final_state = apply_actions(EXAMPLE_SENTENCE, EXAMPLE_ACTIONS, verbose=True)

print('\nfinal tree:')
final_state.root.print()


state after action SHIFT
input: ['me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0']

state after action SHIFT
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0', 'me-1']

state after action ARC_R
input: ['the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0']

state after action SHIFT
input: ['morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0', 'the-2']

state after action SHIFT
input: ['flight-4']
stack: ['ROOT-0', 'Book-0', 'the-2', 'morning-3']

state after action SHIFT
input: []
stack: ['ROOT-0', 'Book-0', 'the-2', 'morning-3', 'flight-4']

state after action ARC_L
input: []
stack: ['ROOT-0', 'Book-0', 'the-2', 'flight-4']

state after action ARC_L
input: []
stack: ['ROOT-0', 'Book-0', 'flight-4']

state after action ARC_R
input: []
stack: ['ROOT-0', 'Book-0']

state after action ARC_R
input: []
stack: ['ROOT-0']

final tree:
ROOT-0
  --Book-0
    --me-1
    --flight-4
      --morning-3
      --the-2


One thing we've ignored above is that some actions may be impossible. For example, we can't shift when there are no words left in the input. Likewise, we can't do `ARC_L` or `ARC_R` if there are fewer than two elements on the stack.

Implement the method below to check for these cases. The method will return a valid action string if the given action is invalid. 

If `SHIFT` is illegal, return `ARC_R`.

If `ARC_L` or `ARC_R` is illegal, return `SHIFT`.

If no moves are legal, raise an exception `raise Exception('no valid action`)`.

In [35]:
def check_action(state, action):
    # YOUR CODE HERE
    if action == 'SHIFT':
        if len(state.input)==0:
            if len(state.stack)>=2:
                return str('ARC_R')
            else:
                raise Exception('no valid action: %s' % action)
        else:
            return action
    elif action == 'ARC_L' or action == 'ARC_R' :
        # YOUR CODE HERE
        if len(state.stack) < 2:
            if len(state.input)!=0:
                return str('SHIFT')
            else:
                raise Exception('no valid action: %s' % action)
        else:
            return action
    else:
        raise Exception('no valid action: %s' % action)    
        
    #raise NotImplementedError()
        
state = State(['Hi', 'there'])
print(state)

print('\ncheck_action', check_action(state, 'SHIFT'))
apply_action(state, 'SHIFT')
print(state)

print('\ncheck_action', check_action(state, 'SHIFT'))
apply_action(state, 'SHIFT')
print(state)

print('\ncheck_action', check_action(state, 'SHIFT'))
apply_action(state, 'ARC_R')
print(state)


input: ['Hi-0', 'there-1']
stack: ['ROOT-0']

check_action SHIFT
input: ['there-1']
stack: ['ROOT-0', 'Hi-0']

check_action SHIFT
input: []
stack: ['ROOT-0', 'Hi-0', 'there-1']

check_action ARC_R
input: []
stack: ['ROOT-0', 'Hi-0']


In [36]:
state = State(['Hi', 'there'])
print(state)

print('\ncheck_action', check_action(state, 'SHIFT'))
assert check_action(state, 'SHIFT') == 'SHIFT'
apply_action(state, 'SHIFT')
print(state)


print('\ncheck_action', check_action(state, 'SHIFT'))
assert check_action(state, 'SHIFT') == 'SHIFT'
apply_action(state, 'SHIFT')
print(state)


print('\ncheck_action', check_action(state, 'SHIFT'))
assert check_action(state, 'SHIFT') == 'ARC_R'
apply_action(state, 'ARC_R')
print(state)



input: ['Hi-0', 'there-1']
stack: ['ROOT-0']

check_action SHIFT
input: ['there-1']
stack: ['ROOT-0', 'Hi-0']

check_action SHIFT
input: []
stack: ['ROOT-0', 'Hi-0', 'there-1']

check_action ARC_R
input: []
stack: ['ROOT-0', 'Hi-0']


## Training an action classifier 

OK!  We now have some infrastructure in place to apply a sequence of actions to create a dependency tree.

Next, we'll need to train a classifier to predict the best action at each time step. To do this, we'll use some labeled data. Each example contains a sentence and the sequence of true actions to get the correct dependency tree.


In [37]:
def read_data(fname):
    data = []    
    for line in open(fname):
        sentence, actions = line.split("\t")
        sentence = sentence.strip().split()
        actions = actions.strip().split()
        data.append({'sentence': sentence, 'actions': actions})
    return data

train_data = read_data(os.path.join('data', 'train.txt'))

print('read %d sentences' % len(train_data))

print('here is the fifth example')
train_data[4]

read 10001 sentences
here is the fifth example


{'sentence': ['The',
  'third',
  'was',
  'being',
  'run',
  'by',
  'the',
  'head',
  'of',
  'an',
  'investment',
  'firm',
  '.'],
 'actions': ['SHIFT',
  'SHIFT',
  'ARC_L',
  'SHIFT',
  'SHIFT',
  'SHIFT',
  'ARC_L',
  'ARC_L',
  'ARC_L',
  'SHIFT',
  'SHIFT',
  'SHIFT',
  'ARC_L',
  'ARC_L',
  'SHIFT',
  'SHIFT',
  'SHIFT',
  'SHIFT',
  'ARC_L',
  'ARC_L',
  'ARC_L',
  'ARC_R',
  'ARC_R',
  'SHIFT',
  'ARC_R',
  'ARC_R']}

In [38]:
# and here we can use apply_actions apply the true actions to get the final dependency tree
final_state = apply_actions(train_data[4]['sentence'], train_data[4]['actions'], verbose=False)
final_state.root.print()

ROOT-0
  --run-4
    --being-3
    --was-2
    --third-1
      --The-0
    --head-7
      --the-6
      --by-5
      --firm-11
        --investment-10
        --an-9
        --of-8
    --.-12


### Generate training data

To generate training examples, we will loop over each example, then apply each true action.

At each step, we will yield a `(state, action)` tuple indicating what the correct action is for that state.

The `state` will be used to compute features, and the `action` will become our class label.

In [39]:
def generate_training_samples(data):
    for d in tqdm(data):
        sentence = d['sentence'].copy()
        actions = d['actions'].copy()
        state = State(sentence)
        for action in actions:
            # we use yield here so this becomes an iterator
            # this can save memory, as opposed to constructing a 
            # giant list before compute the feature vectors.
            yield(state, action)
            apply_action(state, action)

# here is the first training sample generated
state, action = next(generate_training_samples(train_data))
print('state:')
print(state)
print('action:')
print(action)

  0%|          | 0/10001 [00:00<?, ?it/s]

state:
input: ['Al-0', '--1', 'Zaman-2', ':-3', 'American-4', 'forces-5', 'killed-6', 'Shaikh-7', 'Abdullah-8', 'al-9', '--10', 'Ani-11', ',-12', 'the-13', 'preacher-14', 'at-15', 'the-16', 'mosque-17', 'in-18', 'the-19', 'town-20', 'of-21', 'Qaim-22', ',-23', 'near-24', 'the-25', 'Syrian-26', 'border-27', '.-28']
stack: ['ROOT-0']
action:
SHIFT


### Compute features

While we iterate over the training samples, we will compute a list of features over each state.

A feature will be a tuple of (`feature_name, feature_value`). For most features, the `feature_value` will be 1.0.

For example, here is a feature representing the word at the top of the stack (position -1)

We use the special token `EMPTY` to indicate that there is no word present at the given position.

In [40]:
def stack_feature(state):
    if len(state.stack) == 0:
        return ('STACK@-1=EMPTY', 1.0)
    else:
        return ('STACK@-1=%s' % state.stack[-1].word, 1.0)
    
stack_feature(state)

('STACK@-1=ROOT', 1.0)

Similarly, here is a feature representing the word at the front of the input (position 0)

In [41]:
def input_feature(state):
    if len(state.input) == 0:
        return ('INPUT@0=EMPTY', 1.0)
    else:
        return ('INPUT@0=%s' % state.input[0].word, 1.0)
    
input_feature(state)

('INPUT@0=Al', 1.0)

We can now write a function that iterates over all the training samples generated by `generate_training_samples`, computes all feature functions, and stores the result in a list. Along the way, we'll also keep track of the true actions corresponding to each feature list.

Below, we test this for the first two examples in the train_data. As you can see, even with just two sentences, this generates a lot of training examples.

In [42]:
def compute_features(sample_iter, feature_fns):
    features = []
    labels = []
    for state, action in sample_iter:
        features.append([feature_fn(state) for feature_fn in feature_fns])
        labels.append(action)
    return features, labels


feature_fns = [stack_feature, input_feature]
features, labels = compute_features(generate_training_samples(train_data[:2]),
                                    feature_fns)
display(list(zip(labels, features)))

100%|██████████| 2/2 [00:00<00:00, 3052.62it/s]


[('SHIFT', [('STACK@-1=ROOT', 1.0), ('INPUT@0=Al', 1.0)]),
 ('SHIFT', [('STACK@-1=Al', 1.0), ('INPUT@0=-', 1.0)]),
 ('ARC_R', [('STACK@-1=-', 1.0), ('INPUT@0=Zaman', 1.0)]),
 ('SHIFT', [('STACK@-1=Al', 1.0), ('INPUT@0=Zaman', 1.0)]),
 ('ARC_R', [('STACK@-1=Zaman', 1.0), ('INPUT@0=:', 1.0)]),
 ('SHIFT', [('STACK@-1=Al', 1.0), ('INPUT@0=:', 1.0)]),
 ('ARC_R', [('STACK@-1=:', 1.0), ('INPUT@0=American', 1.0)]),
 ('SHIFT', [('STACK@-1=Al', 1.0), ('INPUT@0=American', 1.0)]),
 ('SHIFT', [('STACK@-1=American', 1.0), ('INPUT@0=forces', 1.0)]),
 ('ARC_L', [('STACK@-1=forces', 1.0), ('INPUT@0=killed', 1.0)]),
 ('SHIFT', [('STACK@-1=forces', 1.0), ('INPUT@0=killed', 1.0)]),
 ('ARC_L', [('STACK@-1=killed', 1.0), ('INPUT@0=Shaikh', 1.0)]),
 ('SHIFT', [('STACK@-1=killed', 1.0), ('INPUT@0=Shaikh', 1.0)]),
 ('SHIFT', [('STACK@-1=Shaikh', 1.0), ('INPUT@0=Abdullah', 1.0)]),
 ('ARC_R', [('STACK@-1=Abdullah', 1.0), ('INPUT@0=al', 1.0)]),
 ('SHIFT', [('STACK@-1=Shaikh', 1.0), ('INPUT@0=al', 1.0)]),
 ('ARC_R

Let's write a few more feature functions. Complete the methods below, again using `EMPTY` to indicate that there is no word present at the given position.

In [43]:
def stack_feature_2(state):
    """
    Return a feature for the second-to-last element of the stack.
    e.g., (STACK@-2=years, 1.0)
    """
    # YOUR CODE HERE
    if len(state.stack) < 2:
        return ('STACK@-2=EMPTY', 1.0)
    else:
        return ('STACK@-2=%s' % state.stack[-2].word, 1.0)
    
    #raise NotImplementedError()

def input_feature_2(state):
    """
    Return a feature for the second element of the input.
    e.g., (INPUT@1=years, 1.0)
    """
    # YOUR CODE HERE
    if len(state.input) < 2:
        return ('INPUT@1=EMPTY', 1.0)
    else:
        return ('INPUT@1=%s' % state.input[1].word, 1.0)
    #raise NotImplementedError()

def stack_pair_feature(state):
    """
    A combination of the last and second-to-last elements of the stack.
    e.g., STACK@-1=being__STACK@-2=was
    """
    # YOUR CODE HERE
    f1=stack_feature(state)[0]
    f2=stack_feature_2(state)[0]
    return ('%s__%s' % (f1, f2), 1.0)
    #raise NotImplementedError()

In [44]:
state = State(EXAMPLE_SENTENCE)
print(state)
print(stack_feature(state))
assert stack_feature(state) == ('STACK@-1=ROOT', 1.0)
print(stack_feature_2(state))
assert stack_feature_2(state) == ('STACK@-2=EMPTY', 1.0)
print(input_feature(state))
assert input_feature(state) == ('INPUT@0=Book', 1.0)
print(input_feature_2(state))
assert input_feature_2(state) == ('INPUT@1=me', 1.0)
print(stack_pair_feature(state))
assert stack_pair_feature(state) == ('STACK@-1=ROOT__STACK@-2=EMPTY', 1.0)
apply_action(state, 'SHIFT')

print()
print('state after SHIFT')
print(state)
print(stack_feature(state))
print(stack_feature_2(state))
print(input_feature(state))
print(input_feature_2(state))
print(stack_pair_feature(state))
assert stack_pair_feature(state) == ('STACK@-1=Book__STACK@-2=ROOT', 1.0)

input: ['Book-0', 'me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0']
('STACK@-1=ROOT', 1.0)
('STACK@-2=EMPTY', 1.0)
('INPUT@0=Book', 1.0)
('INPUT@1=me', 1.0)
('STACK@-1=ROOT__STACK@-2=EMPTY', 1.0)

state after SHIFT
input: ['me-1', 'the-2', 'morning-3', 'flight-4']
stack: ['ROOT-0', 'Book-0']
('STACK@-1=Book', 1.0)
('STACK@-2=ROOT', 1.0)
('INPUT@0=me', 1.0)
('INPUT@1=the', 1.0)
('STACK@-1=Book__STACK@-2=ROOT', 1.0)


In [45]:
# here are the new features for the first 5 instances.
feature_fns = [stack_feature, input_feature, stack_feature_2, input_feature_2, stack_pair_feature]
features, labels = compute_features(generate_training_samples(train_data[:2]),
                                    feature_fns)
display(list(zip(labels, features))[:5])

100%|██████████| 2/2 [00:00<00:00, 2129.63it/s]


[('SHIFT',
  [('STACK@-1=ROOT', 1.0),
   ('INPUT@0=Al', 1.0),
   ('STACK@-2=EMPTY', 1.0),
   ('INPUT@1=-', 1.0),
   ('STACK@-1=ROOT__STACK@-2=EMPTY', 1.0)]),
 ('SHIFT',
  [('STACK@-1=Al', 1.0),
   ('INPUT@0=-', 1.0),
   ('STACK@-2=ROOT', 1.0),
   ('INPUT@1=Zaman', 1.0),
   ('STACK@-1=Al__STACK@-2=ROOT', 1.0)]),
 ('ARC_R',
  [('STACK@-1=-', 1.0),
   ('INPUT@0=Zaman', 1.0),
   ('STACK@-2=Al', 1.0),
   ('INPUT@1=:', 1.0),
   ('STACK@-1=-__STACK@-2=Al', 1.0)]),
 ('SHIFT',
  [('STACK@-1=Al', 1.0),
   ('INPUT@0=Zaman', 1.0),
   ('STACK@-2=ROOT', 1.0),
   ('INPUT@1=:', 1.0),
   ('STACK@-1=Al__STACK@-2=ROOT', 1.0)]),
 ('ARC_R',
  [('STACK@-1=Zaman', 1.0),
   ('INPUT@0=:', 1.0),
   ('STACK@-2=Al', 1.0),
   ('INPUT@1=American', 1.0),
   ('STACK@-1=Zaman__STACK@-2=Al', 1.0)])]

Expected output:

```
[('SHIFT',
  [('STACK@-1=ROOT', 1.0),
   ('INPUT@0=Al', 1.0),
   ('STACK@-2=EMPTY', 1.0),
   ('INPUT@1=-', 1.0),
   ('STACK@-1=ROOT__STACK@-2=EMPTY', 1)]),
 ('SHIFT',
  [('STACK@-1=Al', 1.0),
   ('INPUT@0=-', 1.0),
   ('STACK@-2=ROOT', 1.0),
   ('INPUT@1=Zaman', 1.0),
   ('STACK@-1=Al__STACK@-2=ROOT', 1)]),
 ('ARC_R',
  [('STACK@-1=-', 1.0),
   ('INPUT@0=Zaman', 1.0),
   ('STACK@-2=Al', 1.0),
   ('INPUT@1=:', 1.0),
   ('STACK@-1=-__STACK@-2=Al', 1)]),
 ('SHIFT',
  [('STACK@-1=Al', 1.0),
   ('INPUT@0=Zaman', 1.0),
   ('STACK@-2=ROOT', 1.0),
   ('INPUT@1=:', 1.0),
   ('STACK@-1=Al__STACK@-2=ROOT', 1)]),
 ('ARC_R',
  [('STACK@-1=Zaman', 1.0),
   ('INPUT@0=:', 1.0),
   ('STACK@-2=Al', 1.0),
   ('INPUT@1=American', 1.0),
   ('STACK@-1=Zaman__STACK@-2=Al', 1)])]
```

### Fit classifier

To train the classifier, we'll first compute all the features, then use [DictVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html) to create a sparse feature matrix.

In [46]:
features, labels = compute_features(
    generate_training_samples(train_data),
    [stack_feature, stack_feature_2, stack_pair_feature, input_feature, input_feature_2])
vec = DictVectorizer()
X_train = vec.fit_transform(dict(f) for f in features)
X_train.shape

100%|██████████| 10001/10001 [00:02<00:00, 4312.69it/s]


(341552, 224483)

In [47]:
clf = LogisticRegression(max_iter=100, class_weight='balanced')
clf.fit(X_train, labels)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight='balanced')

In [48]:
# print top coefficients for each class.
feats = npa(vec.get_feature_names())
for ci, cname in enumerate(clf.classes_):
    print(cname)
    theta = clf.coef_[ci]
    for i in np.argsort(theta)[::-1][:15]:
        print(feats[i], theta[i])

ARC_L
INPUT@0=EMPTY 14.524752345198065
STACK@-2=very 5.749726444062386
STACK@-2=There 5.490462562135689
STACK@-2=, 5.339117916469915
STACK@-2=for 5.218886682806136
STACK@-2=with 4.852941198002602
STACK@-2=I 4.783636518915244
STACK@-2=a 4.648052015707228
STACK@-2=to 4.486867340963744
STACK@-2=It 4.434911564979165
STACK@-2=/ 4.354319311404713
STACK@-2=from 4.281199744114217
STACK@-2=the 4.2444744722911585
STACK@-2=or 4.040197645179698
STACK@-2=by 3.9891510841024367
ARC_R
INPUT@0=EMPTY 15.853561675978884
STACK@-1=that__STACK@-2=so 6.200443574856145
STACK@-1=of__STACK@-2=because 6.119218690841352
STACK@-1=, 5.37541983010691
STACK@-1=course__STACK@-2=of 5.064098349705973
STACK@-1=? 5.033751414205209
STACK@-1=: 4.932816102510562
STACK@-1=as__STACK@-2=as 4.707892838417774
STACK@-1=! 4.2873600801444365
STACK@-1=s 4.037135091787385
STACK@-1=as__STACK@-2=such 4.0125661209749275
STACK@-1=well__STACK@-2=as 3.9895687946077243
STACK@-1=" 3.978935333197669
STACK@-1=; 3.9152903410641375
STACK@-1=- 3.8

**SHORT ANSWER**: 

**Look at the top coefficients for each class. Do any of the top features make sense to you? Pick two features and explain why you think they are highly weighted, along with examples of each. Enter your answer below.**

For the 'SHIFT' class, STACK@-2=ROOT, STACK@-1=ROOT__STACK@-2=EMPTY, STACK@-2=EMPTY, STACK@-1=ROOT appear at the top features, since they indicate the start of one sentence, they need 'SHIFT' operation to push another work into the stack. For 'ARC_R' class, there are lots of punctuations, which represent the end of one sentence, they need a 'ARC_R' action to produce a head-dependent relation and all of the dependent words at the top of the stack have been assigned, which usually happend at the end of a list of actions. Also, I noticed INPUT@0=EMPTY is the top 1 feature in either 'ARC_L' or 'ARC_R', because it represents the input is empty and no ‘SHIFT’ action allowed.

## Using the classifier to parse

With the classifier `clf`, we can now predict a good action to take for any possible state.

To do so, we'll create a new class called `ClassifierParser`, which will apply the lcassifier to parse a given sentence.

For any given state, we'll need to (i) compute the feature functions, (ii) transform the feature functions into a sparse vector, (iii) call `clf.predict` to get the most probable action for the given state.

In [49]:
class ClassifierParser:
    def __init__(self, clf, vec, feature_fns):
        self.clf = clf
        self.vec = vec
        self.feature_fns = feature_fns
        
    def compute_features(self, state):
        """
        Apply all of the `feature_fns` to this state, then
        call `vec.transform` to convert them into a csr_matrix with a single row.
        Return this matrix.
        """
        features = []
        features.append([feature_fn(state) for feature_fn in feature_fns])
        #vec = DictVectorizer()
        X_test = self.vec.transform(dict(f) for f in features) 
        return X_test
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def predict_action(self, state):  
        """
        First, call `compute_features` to get the feature matrix for this single instance.
        Then, return the predicted action for this sequence.
        The returned value should be a single string in ['SHIFT', 'ARC_L', 'ARC_R']
        """
        # YOUR CODE HERE
        x_test=self.compute_features(state)
        pred=clf.predict(x_test)
        return pred
        #raise NotImplementedError()
        
    def parse(self, sentence, verbose=False):
        """
        Putting it all together, here we will:
        - Create an initial State for this sentence
        - while the state is not finished:
          + call `predict_action` to get the next action to take
          + call `check_action` to ensure the action is valid
          + apply the action
        
        Return: a tuple:
          state.....the final State at the end of parsing
          actions...the list of actions taken
        """
        # YOUR CODE HERE
        state_test = State(sentence)
        actions=[]
        #i=0
        while is_finished(state_test) == False:
#               i+=1
#               print(i)
            pred=self.predict_action(state_test)
            che_act=check_action(state_test, pred)
            #print(check_action(state_test, pred))
            apply_action(state_test, che_act)
            actions.append(che_act[0])
#               print(state_test)
#               print(actions)
            
        return state_test, actions
        #raise NotImplementedError()
    
# we don't get this example exactly right
parser = ClassifierParser(clf, vec, feature_fns)
state, actions = parser.parse(EXAMPLE_SENTENCE)
print('\npredicted actions')
print(actions)
print('\ntrue actions')
print(EXAMPLE_ACTIONS)

print('\npredicted tree')
state.root.print()
print('\ntrue tree')
apply_actions(EXAMPLE_SENTENCE, EXAMPLE_ACTIONS, verbose=False).root.print()


predicted actions
['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'SHIFT', 'ARC_R', 'ARC_R']

true actions
['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_R', 'ARC_R']

predicted tree
ROOT-0
  --morning-3
    --the-2
    --Book-0
      --me-1
    --flight-4

true tree
ROOT-0
  --Book-0
    --me-1
    --flight-4
      --morning-3
      --the-2


Expected output (some variations expected given slightly different classifier parameters)

```
predicted actions
['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'SHIFT', 'ARC_R', 'ARC_R']

true actions
['SHIFT', 'SHIFT', 'ARC_R', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_R', 'ARC_R']

predicted tree
ROOT-0
  --morning-3
    --the-2
    --Book-0
      --me-1
    --flight-4

true tree
ROOT-0
  --Book-0
    --me-1
    --flight-4
      --morning-3
      --the-2
```

In [50]:
state, actions = parser.parse(train_data[4]['sentence'])
print('\npredicted actions')
print(actions)
print('\ntrue actions')
print(train_data[4]['actions'])

print('\npredicted tree')
state.root.print()
print('\ntrue tree')
apply_actions(train_data[4]['sentence'], train_data[4]['actions'], verbose=False).root.print()


predicted actions
['SHIFT', 'SHIFT', 'ARC_L', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_L', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_L', 'ARC_R', 'ARC_R', 'SHIFT', 'ARC_R', 'ARC_R']

true actions
['SHIFT', 'SHIFT', 'ARC_L', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_L', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'ARC_L', 'ARC_L', 'ARC_L', 'ARC_R', 'ARC_R', 'SHIFT', 'ARC_R', 'ARC_R']

predicted tree
ROOT-0
  --run-4
    --being-3
    --was-2
    --third-1
      --The-0
    --head-7
      --the-6
      --by-5
      --firm-11
        --investment-10
        --an-9
        --of-8
    --.-12

true tree
ROOT-0
  --run-4
    --being-3
    --was-2
    --third-1
      --The-0
    --head-7
      --the-6
      --by-5
      --firm-11
        --investment-10
        --an-9
        --of-8
    --.-12


## Evaluation

To measure the accuracy of the parsers, we will first read in the test data, then generate parses for each sentence.
Using our `get_parents` method from above, we'll then compare the parent links in the predicted and true trees.

Our measure will be the fraction of predicted parent links that are present in the true tree.

E.g., if the predicted tree contains edges a->b, b->c, c->d, but the true tree has edges a->b, b->c, b->d, then the score will be 2/3.

We'll compute the accuracy by iterating through every tree and summing together the numerator and denominator.


In [51]:
test_data = read_data(os.path.join('data', 'test.txt'))
print('read %d test sentences' % len(test_data))

read 501 test sentences


In [52]:
def evaluate(parser, test_data):
    """
    Evaluate the parser on this test data using the instructions above.
    Return:
      a float indicating the edge accuracy.
    """
    # YOUR CODE HERE
    denominator_sum=0
    numerator_sum=0
    for data in test_data:
        #prediction
        state, actions = parser.parse(data['sentence'])
        node2parent_predict = {}
        parents_predict =get_parents(state.root, node2parent_predict)
        
        #ground truth
        temp=apply_actions(data['sentence'], data['actions'], verbose=False).root
        node2parent_test = {}
        parents_test =get_parents(temp, node2parent_test)
        
        #calculate how many equal pairs
        common_pairs=0
        for key in parents_test:
            if (key in parents_predict and parents_test[key] == parents_predict[key]):
                common_pairs+= 1
        denominator_sum+=max(len(parents_predict),len(parents_test))
        numerator_sum+=common_pairs
    ratio=numerator_sum/denominator_sum
    return ratio
    #raise NotImplementedError()

evaluate(parser, test_data)
# I get around 0.58

0.5813454499558248

In [53]:
# create an oracle classifier for testing the evaluation method on the first test example.
# it should have an accuracy of 1.0, since it just applies the true actions.
class OracleParser:
    def __init__(self, true_actions):
        self.true_actions = true_actions
        
    def parse(self, sentence):
        return apply_actions(sentence, self.true_actions), self.true_actions       
        
assert evaluate(OracleParser(test_data[0]['actions']), test_data[:1]) == 1.0


## Bonus: Feature engineering

We now have a working dependency parser. The accuracy is actually not too bad, given how few features we're using.

For 5 extra points, spend some time on feature engineering and parameter tuning to see if you can improve accuracy. You may want to iterate on a somewhat smaller version of the dataset to save time before applying to the larger dataset. Explain what you did, how much of a difference it made, and why you think it helped.



Summary:
- add input_pair_feature(a combination of the first and second elements of the input), accuracy=0.5789473684210527
- add stack_feature_i1(a feature for the first element of the stack), accuracy=0.5427237157642307
- remove stack_pair_feature, accuracy=0.5723841979048341
- remove input_feature_2, accuracy=0.5954815095292187 --> best accuracy I can get
- remove stack_feature_2, accuracy=0.06550549034456646

In [54]:
def input_pair_feature(state):
    """
    A combination of the first and second elements of the input.
    e.g., INPUT@0=being__INPUT@1=was
    """
    # YOUR CODE HERE
    f1=input_feature(state)[0]
    f2=input_feature_2(state)[0]
    return ('%s__%s' % (f1, f2), 1.0)
    #raise NotImplementedError()

In [55]:
features, labels = compute_features(
    generate_training_samples(train_data),
    [stack_feature, stack_feature_2, stack_pair_feature, input_feature, input_feature_2,input_pair_feature])
vec = DictVectorizer()
X_train = vec.fit_transform(dict(f) for f in features)
X_train.shape
clf = LogisticRegression(max_iter=100, class_weight='balanced')
clf.fit(X_train, labels)
parser = ClassifierParser(clf, vec, feature_fns)
evaluate(parser, test_data)

100%|██████████| 10001/10001 [00:04<00:00, 2497.65it/s]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5789473684210527

In [56]:
def stack_feature_i1(state):
    """
    Return a feature for the first element of the stack.
    e.g., (STACK@0=years, 1.0)
    """
    # YOUR CODE HERE
    if len(state.stack) != 0:
        return ('STACK@0=EMPTY', 1.0)
    else:
        return ('STACK@0=%s' % state.stack[0].word, 1.0)
    
    #raise NotImplementedError()

In [57]:
features, labels = compute_features(
    generate_training_samples(train_data),
    [stack_feature, stack_feature_2, stack_pair_feature, input_feature, input_feature_2,stack_feature_i1])
vec = DictVectorizer()
X_train = vec.fit_transform(dict(f) for f in features)
X_train.shape
clf = LogisticRegression(max_iter=100, class_weight='balanced')
clf.fit(X_train, labels)
parser = ClassifierParser(clf, vec, feature_fns)
evaluate(parser, test_data)

100%|██████████| 10001/10001 [00:02<00:00, 3828.49it/s]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5427237157642307

In [58]:
features, labels = compute_features(
    generate_training_samples(train_data),
    [stack_feature, stack_feature_2, input_feature, input_feature_2])
vec = DictVectorizer()
X_train = vec.fit_transform(dict(f) for f in features)
X_train.shape
clf = LogisticRegression(max_iter=100, class_weight='balanced')
clf.fit(X_train, labels)
parser = ClassifierParser(clf, vec, feature_fns)
evaluate(parser, test_data)

100%|██████████| 10001/10001 [00:01<00:00, 5349.67it/s]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5723841979048341

In [59]:
features, labels = compute_features(
    generate_training_samples(train_data),
    [stack_feature, stack_feature_2, input_feature, stack_pair_feature])
vec = DictVectorizer()
X_train = vec.fit_transform(dict(f) for f in features)
X_train.shape
clf = LogisticRegression(max_iter=100, class_weight='balanced')
clf.fit(X_train, labels)
parser = ClassifierParser(clf, vec, feature_fns)
evaluate(parser, test_data)

100%|██████████| 10001/10001 [00:02<00:00, 4971.55it/s]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.5954815095292187

In [60]:
features, labels = compute_features(
    generate_training_samples(train_data),
    [stack_feature, input_feature_2, input_feature, stack_pair_feature])
vec = DictVectorizer()
X_train = vec.fit_transform(dict(f) for f in features)
X_train.shape
clf = LogisticRegression(max_iter=100, class_weight='balanced')
clf.fit(X_train, labels)
parser = ClassifierParser(clf, vec, feature_fns)
evaluate(parser, test_data)

100%|██████████| 10001/10001 [00:01<00:00, 5345.65it/s]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.06550549034456646